In [2]:
%matplotlib inline
import tensorflow as tf
from vahun.corpus import TSV_Corpus as Corpus
import numpy as np
from vahun.tools import Timer
from vahun.tools import explog
#from vahun.autoencoder import Autoencoder_ffnn
from vahun.tools import show_performance
from vahun.genetic import Settings
from vahun.tools import get_reconstruction

from vahun.Autoencoder_FFNN import Autoencoder_FFNN
from vahun.Autoencoder_Variational import Autoencoder_Variational
timer=Timer()



corpus_path='/mnt/store/velkey/mnsz2/brain_split.200k.maxlen20'
encode=100
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Xcorpus=Corpus(corpus_path=corpus_path,col=0,size=0)
Ycorpus=Corpus(corpus_path=corpus_path,col=1,size=0)

logger=explog(encoder_type="demo_autoencoder_splitbrain_"+str(encode),
              encoding_dim=encode,
              feature_len=20,
              lang=corpus_path,
              unique_words=len(set(Xcorpus.wordlist)),
              name="demo_autoencoder_top_splitbrain_"+str(encode),
              population_size=0,
              words=len(Xcorpus.wordlist))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
encoder=Autoencoder_FFNN(
                 logger=logger,tf_session=sess,
                 inputdim=len(Xcorpus.abc)*20,
                 layerlist=[encode,len(Xcorpus.abc)*20],
                 encode_index=1,corpus=Xcorpus,
                 optimizer =tf.train.AdamOptimizer(learning_rate = 0.001),
                 nonlinear=tf.sigmoid,disp_step=100,
                 charnum=len(Xcorpus.abc))

In [4]:
encoder.train(Xcorpus.x_train,Xcorpus.x_valid,Xcorpus.x_test,
              512,30,
              Ycorpus.x_train,Ycorpus.x_valid,Ycorpus.x_test)

memoryerror during train reconstruction,using only the half data!


In [6]:
result=encoder.get_reconstruction_splitbrain(Xcorpus.x_test,Xcorpus,Ycorpus.x_test)

In [7]:
with open('/mnt/store/velkey/splitbrain_200_encoding', "a") as myfile:
    for it in result:
        string=""
        for i in it:
            string+=str(i)
            string+='\t'
        string+='\n'
        myfile.write(string)

In [8]:
for i in range(30):
    print(result[i][3],'\t',result[i][4])

           túlhaladt 	            talhatatt
           túlhaladt 	            súlhaledk
        újdonságának 	         úldantásánal
        újdonságának 	         ejeosszgának
               akkus 	                aikas
               akkus 	                okkuk
          tokozásban 	           takszásbal
          tokozásban 	           korolásban
          kezdhettem 	           kszehetnek
          kezdhettem 	           keldtettem
            vásárnak 	             vasornak
            vásárnak 	             kálásnak
       járnak-kelnek 	        jereal-teknek
       járnak-kelnek 	        kálnakekelnek
            örvénybe 	             ölveneba
            örvénybe 	             erményee
           megyéknél 	            megyésnel
           megyéknél 	            menyektét
               chili 	                csini
               chili 	                lheló
               bújna 	                bajra
               bújna 	                kúlni
           kiújulása 	          

In [10]:
std=show_performance(encoder,["eh","kecske","kutya","aytuk","macska","árvíztűrő","fúró","kacsa","a","és"],Xcorpus,printer=True,inputfsize=len(Xcorpus.abc))

ValueError: cannot reshape array of size 980 into shape (10,49)

In [ ]:
def decode_critical(lista,enc=180):
    for POS in lista:
        encoded=np.ones(enc)*-10
        encoded[POS]=10
        a=encoder.decode([encoded])[0].reshape([10,len(corpus.abc)])
        b=corp.defeaturize_data_charlevel_onehot([a])
        print(b)

In [ ]:
import heapq
topstd=heapq.nlargest(6, range(len(stds)), stds.__getitem__)
big_stuff=[]
for i in range(50):
    big_stuff.append([])
    for topind in topstd:
        a=encoder.encode([corpus.x_test[i]])
        a[0][topind]=2
        b=encoder.decode([a[0]])[0].reshape([10,len(corpus.abc)])
        c=corpus.defeaturize_data_charlevel_onehot([b])
        big_stuff[i].append(c)
for row in big_stuff:
    print(row)